## Running a HuggingFace model locally

**You will need to clone this GitHub repository or download this Jupyter Notebook to run locally.**
 

In this notebook we are going to learn how to run a Hugging Face model locally. If you don't have a GPU in your local machine (laptop), that is still fine; you can run the final piece which shows a llama.cpp implementation. Please refer to the README file for more information regarding the python installations and llama.cpp.

Pros of HuggingFace Transformers library:
* downloads the models automatically
* there are snippets of code available to run any model
* easily intergratable models to your project

Cons:
* you need to code the behaviour / user interaction
* it's not as fast as other alternatives and fails to run moderate sized LLMs (>3-4B parameters)
* large computational resources

#### Install Dependencies

In [ ]:
!pip install torchvision==0.17.2
!pip install -U torch==2.3.1
!pip install tensorflow==2.18.0
!pip install transformers==4.47.0
!pip install datasets==3.2.0
!pip install ipywidgets==8.1.5
!pip install tf-keras

#### Importing packages and checking GPU access

In [2]:
# Import some basic packages
import os
import torch 
import torchvision
import tensorflow as tf
import pandas as pd

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoModelForCausalLM
from datasets import load_dataset

from huggingface_hub import hf_hub_download
from huggingface_hub import login

In [4]:
# make sure you have an updated transformers version (>4.43)
import transformers
print(transformers.__version__)


4.47.0


In [6]:
# Both should return 'True' if running on MacOs
print(torch.backends.mps.is_available())
print(torch.backends.mps.is_built())
# Should recognize 1 GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

True
True
Num GPUs Available:  0


In [4]:
# Enable the GPU
DEVICE = "mps" if torch.backends.mps.is_available() else "cpu"
DEVICE

'mps'

#### Authenticate through HuggingFace

HuggingFace requires to login and use a specialised token to use most of their models and datasets. We suggest creating a HuggingFace account and going to settings to create an access token. Use a name of your prefrerence and create either a read or a write token. Useful video walkthrough on how to generate tokens:https://www.youtube.com/watch?v=Br7AcznvzSA . Try to not share your token when sharing this notebook. 

In [12]:
# Login to HuggingFace with your token
from huggingface_hub import notebook_login
notebook_login()

# Alternatively, you can login via terminal with this command: huggingface-cli login 
# Add your token and hit Y when prompted to add token as git credentials

#### Load and run HuggingFace models (non-LLMs)

You need to find a model you'd like to use in HuggingFace and click on the right top corner to use it with HuggingFace. Copy and paste that code block here. Make sure you include a pipeline, an input_text and an output argument to call the model. Some useful examples are provided below to showcase this.

The model name or model_id can be copied from the top part of the page, it will be somethng like 'meta-llama/Llama-2-7b-chat-hf'

In [13]:
torch.mps.empty_cache() # This frees up unused memory

In [ ]:
# Examples of HuggingFace models for different NLP tasks in Investment Analysis

from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

DEVICE = 0  # Use 0 for GPU or -1 for CPU

## Sentiment Analysis for Financial News
# Analyze the sentiment of market news or analyst reports
classifier = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment",
    tokenizer="cardiffnlp/twitter-roberta-base-sentiment",
    device=DEVICE
)

news = "The stock market rallied today after the Federal Reserve's announcement."
results = classifier(news)
print(f"Sentiment: {results[0]['label']} with score {results[0]['score']}")

## Named Entity Recognition (NER) for Investment Documents
# Extract named entities such as companies, locations, and financial entities
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer, device=DEVICE)

financial_doc = "Tesla reported strong earnings this quarter, with revenue reaching $21 billion."
ner_results = nlp(financial_doc)
print("Named Entities:")
for entity in ner_results:
    print(f"Entity: {entity['word']}, Label: {entity['entity']}, Score: {entity['score']}")

Device set to use mps
Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps


#### Load and run HuggingFace models (LLMs)

We are going to see some examples of running LLMs using HuggingFace and Langchain below. 

In [8]:
torch.mps.empty_cache() # This frees up unused memory

Example 1: CAUTION! The following is an example that will run OOM (out-of-memory) if run on a ~18GB GPU. We do not advise to run it as it will probably crash your machine applications, that's why we have left it commented out.

In [9]:
# # EXAMPLE of HuggingFace Mistral-7B-v0.1 
# with torch.no_grad():

# # Create a pipeline for text generation or instruction-following    
#     tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
#     model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1")

#     pipe = pipeline("text-generation", model = model, tokenizer = tokenizer, device = DEVICE)     # Load model directly

#     # Provide a financial instruction or a prompt
#     input_text = "Analyze the impact of rising interest rates on global stock markets."
    
#     # Generate the response from the model
#     output = pipe(input_text, max_length=150, truncation=True)
    
#     # Print the result
#     print(output[0]['generated_text'])

Example 2: In the below snippet you will find a sample run of a relatively small LLM. To run an LLM of your choice you need to adapt the code for the tokenizer and model. Also, you need to adjust the
pipeline to the task. Some key tasks:
* "text-generation"
* "text-classification"
* "summarization"
* "question-answering"
* "sentiment-analysis" and more.

In [ ]:
# Example of running an LLM with Langchain
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

# Load the model for financial text generation
hf = HuggingFacePipeline.from_model_id(
    model_id="Qwen/Qwen2.5-1.5B-Instruct", task="text-generation", pipeline_kwargs={"max_new_tokens": 200, "pad_token_id": 50256},
)

from langchain.prompts import PromptTemplate

# Define a prompt template for investment-related questions
template = """Question: {question}

Answer: Let's analyze this step by step."""
prompt = PromptTemplate.from_template(template)

# Create a processing chain
chain = prompt | hf

# Example investment-related question
question = "What are the key differences between ETFs and mutual funds?"

# Generate and print the response
print(chain.invoke({"question": question}))

#### Saving models locally

You can use the following commented out section if you want to save models locally and reload. Models take up a lot of space so be mindful of that, or delete when you don't need them anymore on your local disk.

In [12]:
# model_id = "TheFinAI/finma-7b-full"
# tokenizer = AutoTokenizer.from_pretrained(f"{model_id}", legacy = True)
# tokenizer.save_pretrained(f"cache/tokenizer/{model_id}")
# model = AutoModelForCausalLM.from_pretrained(f"{model_id}")
# model.save_pretrained(f"cache/model/{model_id}")
# tokenizer = AutoTokenizer.from_pretrained(f"./{model_id}", legacy=True)
# model = AutoModelForCausalLM.from_pretrained(f"./{model_id}")

#### Note on the LLMs inference behaviour

You can fine-tune the behaviour of the generation by adjusting parameters such as:

* max_length: The maximum length of the generated response in number of tokens.
* temperature: Affects randomness in the output (lower values make output more deterministic, while higher values produce more creative responses.
* top_k: Limits the sampling pool to the top k most likely tokens at each generation step.
* top_p: Filters the sampling pool based on cumulative probability, ensuring only the most probable tokens with a combined probability of p are considered.
* num_return_sequences: Specifies the number of generated outputs to return.


#### Run a model with Langchain


Let's see some advantages and disadvantages of Langchain:
* it uses the HuggingFace Transformer library
* it has access to a lot of models
* based on a Task-Oriented Framework and not ML-oriented as Transformers
* consists of plug-and-play modules

Cons:
* depends on external LLMs (HuggingFace, OpenAI.. )
* not designed for training or fine-tuning models
* large computational resources

In [13]:
# !pip install -U langchain-community

In [14]:
# Example of running an LLM with Langchain
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="microsoft/DialoGPT-medium", task="text-generation", pipeline_kwargs={"max_new_tokens": 200, "pad_token_id": 50256},
)

from langchain.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | hf

question = "What are capital markets?"

print(chain.invoke({"question": question}))

Question: What are capital markets?

Answer: Let's think step by step.Capital markets are a subset of the market.


model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

# LLama CPP

# Option A: 

#### Install Dependencies

In [ ]:
pip install llama-cpp-python==0.3.5

Download the model locally, load it and run inference with the following commands. This Option is for models in HuggingFace that have a GGUF format.

Run the following code command in your MacOs terminal to download a GGUF model of your choice. Adjust the model names accroding to HuggingFace.

In [ ]:
# Log in to HuggingFace and download the model. Please run the following commands from your terminal.
!huggingface-cli login
!huggingface-cli download TheBloke/Llama-2-7b-Chat-GGUF llama-2-7b-chat.Q5_K_S.gguf --local-dir . --local-dir-use-symlinks False


In [6]:
from llama_cpp import Llama

# Load the model
llm = Llama(model_path='/Users/brianpisaneschi/Library/CloudStorage/OneDrive-CFAInstitute/Coding/GitHub/The-Automation-Ahead/Practical Guide For LLMs In the Financial Industry/llama-2-7b-chat.Q5_K_S.gguf')

llama_load_model_from_file: using device Metal (Apple M2 Pro) - 21845 MiB free
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/brianpisaneschi/Library/CloudStorage/OneDrive-CFAInstitute/Coding/GitHub/The-Automation-Ahead/Practical Guide For LLMs In the Financial Industry/llama-2-7b-chat.Q5_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.fe

In [7]:
sentence = "Can you tell me one of the challenges of current capital markets?"
response = llm(sentence, max_tokens=512)
print(response['choices'][0]['text'])

llama_perf_context_print:        load time =    5313.19 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    16 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   448 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   40532.70 ms /   464 tokens




Capital markets are facing a number of challenges, including:

1. Market volatility: Capital markets have experienced increased volatility in recent years, driven by factors such as geopolitical risks, economic uncertainty, and changes in central bank policies.
2. Liquidity crises: The COVID-19 pandemic has exposed weaknesses in the liquidity of some capital markets, particularly in bond markets.
3. Regulatory challenges: Capital market regulators are facing new challenges in the wake of the pandemic, including the need to balance the need for financial stability with the need for economic growth.
4. ESG (Environmental, Social, and Governance) considerations: Increasing investor demand for ESG considerations is driving changes in capital markets, including the development of new financial instruments and the incorporation of ESG factors into investment decisions.
5. Digital transformation: The rapid pace of technological change is transforming capital markets, with new fintech compan

# Option B: 

If you are interested in running a model that only has a HuggingFace format (hf) and not a GGUF, you need to convert it first to GGUF and then lead it in memory and perform inference using the llama.cpp package. To do that, you have to clone the llama.cpp repo. 

Refer to the README file on how to do that step-by-step.

In [ ]:
# Let's suppose we're interested in this model_id: meta-llama/Llama-2-7b-chat-hf
# Download the model locally using the terminal and the command below
# python download.py meta-llama/Llama-2-7b-chat-hf

In [ ]:
# Once the hf model is downloaded, we need to update the convert-hf-to-gguf-update.py in the llama.cpp folder with 
# the new model and tokenizer details

In [ ]:
# Then we need to run: convert-hf-to-gguf-update.py <huggingface_token>
# which will convert the model and save a new model artefact under the gguf file format.

In [18]:
# Using that gguf we conduct inference as above, by loading the gguf model
from llama_cpp import Llama

# Load the model
llm = Llama(model_path='//your_path_to_model/llama.cpp/llama-2-7b-chat.Q5_K_S.gguf')

llama_load_model_from_file: using device Metal (Apple M3 Pro) - 866 MiB free
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from //Users/alexandraspyrou/Library/CloudStorage/OneDrive-CFAInstitute/Code/llama.cpp/llama-2-7b-chat.Q5_K_S.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:

In [20]:
sentence = "Can you tell me one of the challenges of current capital markets?"
response = llm(sentence, max_tokens=512)
print(response['choices'][0]['text'])

Llama.generate: 15 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    4259.35 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   353 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   20735.17 ms /   354 tokens



 literally, one of the challenges that are faced by investors today?

Sure! One of the significant challenges faced by investors in today's capital markets is the increasing volatility and uncertainty caused by the COVID-19 pandemic. The pandemic has disrupted economic activity, led to widespread lockdowns, and resulted in a sharp decline in global trade. This has had a significant impact on financial markets, leading to heightened volatility, lower liquidity, and increased risk premia.
Another challenge facing investors today is the ongoing shift towards sustainable and responsible investing. As investors become more aware of the environmental, social, and governance (ESG) factors that can impact their investments, they are increasingly demanding transparency and accountability from companies regarding their ESG practices. This has led to increased regulatory scrutiny and a greater focus on ESG-related disclosures, which can be challenging for companies to navigate.
Finally, the rise

### Load HuggingFace dataset

As with the models, you need to browse the datasets and find a suitable for your project. The dataset_id can be copied from the top part of the page, it will be somethng like 'TheFinAI/en-fpb'. You can see how to load and convert a dataset in a pandas DataFrame.

Load the dataset (e.g., CICM for stock movement prediction)
Other datasets:
* Sentiment Analysis:
* Question-Answering:
* Summarisation:
* Stock Movement Prediction: 'TheFinAI/flare-sm-cikm', 'TheFinAI/en-fpb', 'TheFinAI/flare-sm-bigdata', 'TheFinAI/flare-ectsum', 'TheFinAI/flare-edtsum'

In [ ]:
# !pip install datasets

In [ ]:
# !pip install -U datasets

In [ ]:
# # Restart the kernel after running this.
# !pip install -U datasets huggingface_hub fsspec


In [12]:
from datasets import load_dataset

# Load the dataset (e.g., IMDB for sentiment analysis)
dataset = load_dataset('TheFinAI/flare-sm-cikm')

README.md:   0%|          | 0.00/651 [00:00<?, ?B/s]

(…)-00000-of-00001-f71a7dda3fae0889.parquet:   0%|          | 0.00/13.3M [00:00<?, ?B/s]

(…)-00000-of-00001-e1663a0932037903.parquet:   0%|          | 0.00/4.15M [00:00<?, ?B/s]

(…)-00000-of-00001-b105ab56855808e4.parquet:   0%|          | 0.00/1.70M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3396 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1143 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/431 [00:00<?, ? examples/s]

In [13]:
# Display the dataset info and a sample
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'query', 'answer', 'text', 'choices', 'gold'],
        num_rows: 3396
    })
    test: Dataset({
        features: ['id', 'query', 'answer', 'text', 'choices', 'gold'],
        num_rows: 1143
    })
    valid: Dataset({
        features: ['id', 'query', 'answer', 'text', 'choices', 'gold'],
        num_rows: 431
    })
})


In [16]:
import pandas as pd

df = pd.DataFrame(dataset['test'])
df.head()

,id,query,answer,text,choices,gold
0,cikmsm3827,Assess the data and tweets to estimate whether...,Fall,"date,open,high,low,close,adj-close,inc-5,inc-1...","[Rise, Fall]",1
1,cikmsm3828,Analyze the information and social media posts...,Rise,"date,open,high,low,close,adj-close,inc-5,inc-1...","[Rise, Fall]",0
2,cikmsm3829,Examine the data and tweets to deduce if the c...,Fall,"date,open,high,low,close,adj-close,inc-5,inc-1...","[Rise, Fall]",1
3,cikmsm3830,Assess the data and tweets to estimate whether...,Rise,"date,open,high,low,close,adj-close,inc-5,inc-1...","[Rise, Fall]",0
4,cikmsm3831,Assess the data and tweets to estimate whether...,Fall,"date,open,high,low,close,adj-close,inc-5,inc-1...","[Rise, Fall]",1
